In [ ]:
from keras import models
from keras import layers
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy
from PIL import Image

# Generator Model

In [ ]:
model1 = models.Sequential()
model1.add(layers.Dense(input_dim=100,output_dim=1024,activation="tanh"))
model1.add(layers.Dense(128*7*7,activation="tanh"))
model1.add(layers.BatchNormalization())
model1.add(layers.Activation("tanh"))
model1.add(layers.Reshape((7,7,128),input_shape=(128*7*7,)))
model1.add(layers.UpSampling2D(size=(2,2)))
model1.add(layers.Conv2D(64,(5,5),padding='same',activation='tanh'))
model1.add(layers.UpSampling2D(size=(2,2)))
model1.add(layers.Conv2D(1,(5,5),padding='same',activation='tanh'))

In [ ]:
help("keras.layers.Conv2D")

# Dsicriminator

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(64,(5,5),padding='same',input_shape=(28,28,1),activation='tanh'))
model2.add(layers.MaxPooling2D(pool_size=(2,2)))
model2.add(layers.Conv2D(128,(5,5),activation='tanh'))
model2.add(layers.MaxPooling2D(pool_size=(2,2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(1024,activation='tanh'))
model2.add(layers.Dense(1,activation='sigmoid'))

# Final Model

In [ ]:
model = models.Sequential()
model.add(model1)
model2.trainable = False
model.add(model2)

# load data

In [ ]:
(trainimg,trainlb),(testimg,testlb) = mnist.load_data()
trainimg = trainimg/255
testimg = testimg/255
trainimg = trainimg.reshape(60000,28,28,1)
testimg = testimg.reshape(10000,28,28,1)

# Combining Images

In [ ]:
def combine_images(g_images):
    num = g_images.shape[0] # getting the count of images
    width = int(numpy.sqrt(num))
    height = int(numpy.ceil(float(num)/width))
    imgshape = g_images[0].shape
    image = numpy.zeros((height*imgshape[0],width*imgshape[1]),dtype = g_images.dtype)
    for ind,img in enumerate(g_images):
        i = int(ind/width)
        j = ind % width
        image[i*imgshape[0]:(i+1)*imgshape[0],i*imgshape[1]:(j+1)*imgshape[1]] = img[:,:,0]
        return image
    
    

# Compilation

In [ ]:
model1_optim = SGD(lr = 0.0005,momentum=0.9,nesterov=True)
model2_optim = SGD(lr = 0.0005,momentum=0.9,nesterov=True)
model1.compile(loss='binary_crossentropy',optimizer="SGD")
model.compile(loss='binary_crossentropy',optimizer=model1_optim)
model2.trainable = True
model2.compile(loss='binary_crossentropy',optimizer=model2_optim)
# training for 100 epochs
for i in range(100):
    print("Epoch is ",i)
    batch_size = 128
    for j in range(int(trainimg.shape[0]/batch_size)):
        noise = numpy.random.uniform(-1,1,size=(batch_size,100))
        img_batch = trainimg[j*batch_size:(j+1)*batch_size]
        g_images = model1.predict(noise,verbose=0)
        if j % 20 ==0:
            img = combine_images(g_images)
            img = img*127.5 + 127.5
            Image.fromarray(img.astype(numpy.uint8)).save(str(i)+"_"+str(j)+".png")
            xtrain = numpy.concatenate((img_batch,g_images))
            ytrain = [1]*batch_size +[0] * batch_size
            model2_loss = model2.train_on_batch(xtrain,ytrain)
            print("batch %d discriminator loss : %f"%(j,model2_loss))
            noise = numpy.random.uniform(-1,1,(batch_size,100))
            model2.trainable = False
            model1_loss = model.train_on_batch(noise,[1]*batch_size)
            model2.trainable = True
            print("batch %d generator loss : %f"%(j,model1_loss))
            if j%10 ==9:
                model1.save_weights("generator",True)
                model2.save_weights("discriminator",True)

# Generation

In [ ]:
model1.compile(loss='binary_crossentropy',optimizer="SGD")
model1.load_weights('generator')
noise = numpy.random.uniform(-1,1,(batch_size,100))
g_iamges2 = model1.predict(noise,verbose=1)
img = combine_images(g_iamges2)
img = img*127.5 + 127.5
Image.fromarray(img.astype(np.uint8)).save("generated_image.png")
Image(img)